# Indicadores
Notebook destinado a codar uma série de indicadores

## Average Directional Index
É um indicador que indica a força da trend
- Abaixo de 20: Muito provável que não exista trend
- Entre 20 e 50: Pode existir uma trend
- Acima de 50: Muito provável que exista uma forte trend

Esse indicador não determina um sinal, apenas diz se existe ou não uma trend

In [12]:
!pip install backtesting

In [14]:
import numpy as numpy
import pandas as pd
from tradingUtils import *
import ta

ModuleNotFoundError: No module named 'backtesting'

In [52]:
def ADX(data, adx_period=14):
    close = data['Adj Close'].values
    open_ = data['Open'].values
    high = data['High'].values
    low = data['Low'].values
    adx = ta.trend.ADXIndicator(high=data['High'], low=data['Low'], close=data['Adj Close'], window=adx_period).adx()
    return adx 

In [53]:
def parabolic_sar(data, acceleration=0.02, max_acceleration=0.2):
    high = data['High'].values
    low = data['Low'].values
    close = data['Adj Close'].values

    sar = ta.trend.PSARIndicator(high=high, low=low, close=close, step=acceleration, max_step=max_acceleration).psar()
    
    return sar


In [54]:
def parabolic_sar(data, acceleration=0.02, max_acceleration=0.2):
    high = data['High'].values
    low = data['Low'].values

    # Initial settings
    sar = []
    af = acceleration
    ep = high[0]  # Initial Extreme Point (assumed high for the first period)
    trend = 1  # Assume starting with an uptrend

    for i in range(len(data)):
        if i == 0:
            sar.append(low[i])  # Initial SAR value is the first low price
            continue

        prev_sar = sar[-1]  # Previous SAR value

        if trend == 1:  # Uptrend
            new_sar = prev_sar + af * (ep - prev_sar)
            sar.append(min(new_sar, low[i-1], low[i]))  # Ensure SAR doesn't cross the lowest points
            if high[i] > ep:
                ep = high[i]  # Update extreme point
                af = min(af + acceleration, max_acceleration)  # Increase AF
            if low[i] < new_sar:  # Trend reversal condition
                trend = -1  # Switch to downtrend
                sar[-1] = ep  # On reversal, set SAR to the last extreme point
                ep = low[i]  # Reset extreme point for the new trend
                af = acceleration  # Reset AF

        else:  # Downtrend
            new_sar = prev_sar + af * (ep - prev_sar)
            sar.append(max(new_sar, high[i-1], high[i]))  # Ensure SAR doesn't cross the highest points
            if low[i] < ep:
                ep = low[i]  # Update extreme point
                af = min(af + acceleration, max_acceleration)  # Increase AF
            if high[i] > new_sar:  # Trend reversal condition
                trend = 1  # Switch to uptrend
                sar[-1] = ep  # On reversal, set SAR to the last extreme point
                ep = high[i]  # Reset extreme point for the new trend
                af = acceleration  # Reset AF

    return pd.Series(sar, index=data.index)


In [55]:
def on_balance_volume(data):
    close = data['Adj Close'].values
    volume = data['Volume'].values
    
    obv = [0]  # Initial OBV starts at 0
    
    for i in range(1, len(close)):
        if close[i] > close[i - 1]:
            obv.append(obv[-1] + volume[i])  # Add volume if price went up
        elif close[i] < close[i - 1]:
            obv.append(obv[-1] - volume[i])  # Subtract volume if price went down
        else:
            obv.append(obv[-1])  # No change if price stayed the same

    return pd.Series(obv, index=data.index)


In [56]:
def average_true_range(data, atr_period=14):
    high = data['High'].values
    low = data['Low'].values
    close = data['Adj Close'].values
    
    true_ranges = [0]  # Start with a true range of 0 for the first value

    for i in range(1, len(close)):
        tr1 = high[i] - low[i]  # High - Low
        tr2 = abs(high[i] - close[i - 1])  # High - Previous Close
        tr3 = abs(low[i] - close[i - 1])  # Low - Previous Close
        true_range = max(tr1, tr2, tr3)  # The maximum of the three is the True Range
        true_ranges.append(true_range)

    # Calculating the ATR using a simple moving average of the true ranges
    atr = pd.Series(true_ranges).rolling(window=atr_period).mean()
    
    return atr


In [57]:
def commodity_channel_index(data, cci_period=20):
    high = data['High'].values
    low = data['Low'].values
    close = data['Adj Close'].values
    
    # Calculate Typical Price
    typical_price = (high + low + close) / 3

    # Calculate moving average of typical price
    tp_sma = pd.Series(typical_price).rolling(window=cci_period).mean()

    # Calculate Mean Deviation
    mean_deviation = pd.Series(typical_price - tp_sma).abs().rolling(window=cci_period).mean()

    # Calculate CCI
    cci = (typical_price - tp_sma) / (0.015 * mean_deviation)
    
    return pd.Series(cci, index=data.index)


In [58]:
def bollinger_bands(data, bb_period=20, num_std=2):
    close = data['Adj Close'].values
    
    # Calculate Simple Moving Average
    sma = pd.Series(close).rolling(window=bb_period).mean()

    # Calculate standard deviation
    std = pd.Series(close).rolling(window=bb_period).std()

    # Upper and Lower Bollinger Bands
    upper_band = sma + (num_std * std)
    lower_band = sma - (num_std * std)

    return pd.DataFrame({
        'SMA': sma,
        'Upper Band': upper_band,
        'Lower Band': lower_band
    }, index=data.index)


In [59]:
data = tsla_data.copy()
run_signal_policy(data, ADX, "ADX", exec_back = False)

data

TypeError: run_signal_policy() got an unexpected keyword argument 'exec_back'